In [1]:
# Scrape odds from odds portal
import time
import requests
import pandas as pd
from datetime import date
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox") # linux only
# chrome_options.add_argument("--headless")
chrome_options.headless = True # also works
PATH = 'C:\\Users\\devna\\Ecotec\\chromedriver.exe'
driver = webdriver.Chrome(PATH, options=chrome_options)

odds_df = ''
odds_list = []

# Start Oct 16, 2018 - June 13, 2019
year = '2019-2020'
pages = 29

# Change timezone to Eastern USA
driver.get('https://www.oddsportal.com/basketball/usa/nba/results/#/page/1/')
time.sleep(3)
timezone = driver.find_elements_by_class_name('user-header-fakeselect')
timezone[1].click()
time.sleep(2)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[7]/div/a[40]').click()


# For loop through all pages
for page in range(1, pages):
    print('Scraping page: ',page)
#     driver.get(f'https://www.oddsportal.com/basketball/usa/nba/results/#/page/{page}/')
    driver.get(f'https://www.oddsportal.com/basketball/usa/nba-{year}/results/#/page/{page}/')

    time.sleep(3)

    table = driver.find_element_by_tag_name('tbody')

    rows = table.find_elements_by_tag_name('tr')

    for row in rows:
        try:
            date = row.find_element_by_css_selector('th.first2.tl').text
        except:
            cols = row.find_elements_by_tag_name('td')

            if len(cols) > 5:
                teams = cols[1].text
                score = cols[2].text
                home_odds = cols[3].text
                vis_odds = cols[4].text
                odds_dict = {
                    'Date': date,
                    'Teams': teams,
                    'Score': score,
                    'Home Odds': home_odds,
                    'Vis Odds': vis_odds
                }
                odds_list.append(odds_dict)



driver.quit()

odds_df = pd.DataFrame(odds_list)
odds_df

Scraping page:  1
Scraping page:  2
Scraping page:  3
Scraping page:  4
Scraping page:  5
Scraping page:  6
Scraping page:  7
Scraping page:  8
Scraping page:  9
Scraping page:  10
Scraping page:  11
Scraping page:  12
Scraping page:  13
Scraping page:  14
Scraping page:  15
Scraping page:  16
Scraping page:  17
Scraping page:  18
Scraping page:  19
Scraping page:  20
Scraping page:  21
Scraping page:  22
Scraping page:  23
Scraping page:  24
Scraping page:  25
Scraping page:  26
Scraping page:  27
Scraping page:  28


,Date,Teams,Score,Home Odds,Vis Odds
0,11 Oct 2020 - Play Offs,,,,
1,11 Oct 2020 - Play Offs,Miami Heat - Los Angeles Lakers\n,93:106,3.01,1.42
2,09 Oct 2020 - Play Offs,,,,
3,09 Oct 2020 - Play Offs,Los Angeles Lakers - Miami Heat\n,108:111,1.31,3.61
4,06 Oct 2020 - Play Offs,,,,
...,...,...,...,...,...
1481,05 Oct 2019 - Pre-season,Golden State Warriors - Los Angeles Lakers,101:123,2.48,1.54
1482,05 Oct 2019 - Pre-season,Indiana Pacers - Sacramento Kings\n,130:106,2.43,1.55
1483,04 Oct 2019 - Pre-season,,,,
1484,04 Oct 2019 - Pre-season,Sacramento Kings - Indiana Pacers\n,131:132 OT,1.48,2.64


In [2]:
# Manipulate the data

# Split Teams into Home and Away
odds_df[['Home','Visitor']] = odds_df['Teams'].str.split("-",expand=True,)
odds_df['Home'] = odds_df['Home'].str.strip().replace('\n', '')
odds_df['Visitor'] = odds_df['Visitor'].str.strip()

# Split Score into home and away
odds_df['Score'] = odds_df['Score'].str.replace('OT', '')
odds_df[['Home PTS', 'Vis PTS']] = odds_df.Score.str.split(':', expand=True)
odds_df

# Drop Teams  and Score column column
odds_df = odds_df.drop(['Teams', 'Score'], axis=1)

# Replace strings from date
odds_df['Date'] = odds_df['Date'].replace(regex=[' - All Stars', ' - Pre-season', ' - Play Offs'], value = '')

# # Remove today and yesterday
# yesterdays_odds = odds_df[odds_df['Date'].str.contains('Yesterday|Today')].index
# to_remove = yesterdays_odds[-1]
# odds_df = odds_df.iloc[to_remove:,:]

# Drop Nan values
odds_df = odds_df.dropna(subset=['Visitor', 'Vis PTS'])

# Convert dates to datetime
odds_df['Date'] = pd.to_datetime(odds_df['Date'])

In [3]:
odds_df

,Date,Home Odds,Vis Odds,Home,Visitor,Home PTS,Vis PTS
1,2020-10-11,3.01,1.42,Miami Heat,Los Angeles Lakers,93,106
3,2020-10-09,1.31,3.61,Los Angeles Lakers,Miami Heat,108,111
5,2020-10-06,3.76,1.30,Miami Heat,Los Angeles Lakers,96,102
7,2020-10-04,4.40,1.23,Miami Heat,Los Angeles Lakers,115,104
9,2020-10-02,1.17,5.43,Los Angeles Lakers,Miami Heat,124,114
...,...,...,...,...,...,...,...
1480,2019-10-05,1.35,3.18,San Antonio Spurs,Orlando Magic,89,125
1481,2019-10-05,2.48,1.54,Golden State Warriors,Los Angeles Lakers,101,123
1482,2019-10-05,2.43,1.55,Indiana Pacers,Sacramento Kings,130,106
1484,2019-10-04,1.48,2.64,Sacramento Kings,Indiana Pacers,131,132


In [4]:
odds_df.to_csv(f'Final_Odds_{year}.csv', index=False)

In [5]:
year

'2019-2020'